# IV/dIdV Analysis Sweep Demo

Author:

Maggie Reed

This is a notebook which should help set you up to practice and understand the SPICE/HERALD IV/dIdV sweep processing and analysis. Sections of this notebook are left blank on purpose, as the notebook is intended to be utilized interactively. This notebook is the practical exercise portion of a series of introductory presentations on the SPICE/HERALD analysis pipeline. Users should be working their way through the appropriate presentation BEFORE using this notebook, as many things are made more clear there. 

While this notebook contains useful theory, students should also be referred to appropriate theses and other readings. Some of the theory covered here is largely introductory, and meant to familiarize a new comer with terms, equations, and parameters. I found Sam Watkins thesis to be very friendly and informative, and of course, for an indepth study, the Bible of TES physics: Kent Irwin's Transition-Edge Sensors paper. Both of these can be found online.  

# Imports, packages

In [2]:
import pandas as pd
import time as tm
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10, 6.5]

from pytesdaq.processing import IVSweepProcessing, IVanalysis
from pytesdaq.processing import _iv_didv_tools_plotting as plot
import qetpy
import pickle

# Setup: channel, data path

In [3]:
# channel
channel = 'Melange1pc1ch'

# path 
save_path = '/data/users/your_user/the_run/iv_process' #This is for saving figures or other files if you want
sweep_data_path = '/sdata2/runs/run32/raw/iv_didv_I2_D20230902_T122408' #the location of the raw unprocessed data


NOTE: 

The raw data is specified according to the following format:

datatype_I2_DYYYYMMDD_THHMMSS | ex: iv_didv_I2_D20230902_T122408

Here we see that the kind of data we are getting is iv_didv data, which was taken on September 2nd, 2023. As we continue in our analysis journey, we will see other kinds of data, such as continuous, random, and trigger data. Our raw data is always taken in HDF5 format. Specifics for this raw data will be discussed later, once we start other types of processing and analysis.

## Preliminary processing of the IV sweep

In [4]:
# instantiate
iv_didv = IVSweepProcessing(sweep_data_path)
iv_didv.describe()

IV/dIdV Sweep: iv_didv_I2_D20230902_T122408
 
Melange1pc1ch:
 -IV: 32 bias points
 -dIdV: 32 bias points


Here we see that our 1% TES device was set up to take data at 32 different bias points. These points were split into three regions for the device: SC, Normal, and Transition. The actual range of these values, their divisions/steps, and the voltage values were all specified by whoever took the raw data. This setup is typically done in a config file. 

Now we will process the raw data. We typically take data with negative QET bias, which is why we flip it via multiplication by 1. This will be depreciated in the future, and the default case when processing will be to multiply by -1 unless specified otherwise.

In [3]:
# process

df = iv_didv.process(channel, 
                     lgc_save=True, save_path=save_path,
                     lgc_output=True, ncores=8)


df.qetbias*= -1 


Once you have processed, be sure to **comment out** the above cell. Processing can take some time, and you do not need to typically reprocess once it is done once. If you want to pick up where you left off, you can reload your data with the below cell.

In [ ]:
# TO LOAD
# df = pd.read_hdf('/data/users/serfass/run32/iv_process/iv_didv_I2_D20230902_T122408/IV_dIdV_processing_D20230912_T142435.hdf',
#                   key=channel)

# Analysis of the IV Sweep

Now we are going to begin our actual analysis. First we want to extract some measured DC characteristics. We will begin with the zero-frequency simplification of the Thevenin equivalent TES circuit:

$V_b = I(R_l + R(T,I)) \label{eq1}\tag{1}$ 

where $V_b$ is the voltage bias ($V_b = I_b*R_{sh}$), $R_l$ is the load resistance ($R_l = R_p + R_{sh}$), and $R(T,I)$ is the TES resistance, where current and temperature dependence has been explicitly shown. 

Lets take $\ref{eq1}$ to two extremes: SC resistance, and normal resistance. 

Starting with SC regime: the relation between the load resistance $R_l$ and shunt resistance $R_{sh}$ is:

$V_b = I_b*R_{sh} = IR_l$

where the bias current ($I_b$) and the shunt resistance ($R_{sh}$) is known. 

Now onto the normal regime:

$V_b = I(R_{l}+R_N)$ 

where $R_N$ is the normal resistance of the TES near the SC transition. 

At this point you should note that both extremes have linear relationships between $V_b$ and $I$. This is why we vary the voltage bias and observe the current change, since we can then extract the unknown parasitic resistance and normal resistance. 

The last region we will consider is the transition region between SC and normal. Here we treat the equilibrium bias power as a constant ($P_0$), and utilizing $P=VI$ on $\ref{eq1}$:
$V_b = IR_l+ IR(T,I) = IR_l+ IR = IR_l + V = IR_l + P_0/I$

Now, we can fit the curves in these regions!

In [6]:
IVobj = IVanalysis(
    df, 
    nnorm=5, # number of points in normal region
    nsc=4, # number of points in superconducting region
    channels=channel,
    lgcremove_badseries=True,
    figsavepath=save_path
)


The number of points in each region is specified when we take the raw data, you can either get them directly from the config files, or you can guess and modify the number after you run the cell below, counting the number of points by eye in each region. The cell above instantiates our IVanalysis object. This object will be built up as we go through our analysis with various parameters saved in a dictionary internally within the object. 

# Fitting parameters

## IV curves

In [5]:
IVobj.rshunt_err = 0.1e-3
IVobj.analyze_sweep(lgcplot=True)

Notice that we specify the shunt error. If you recall, the shunt resistance is a measured quantity, established by the person who fabricated the device. 

**analyze_sweep**: this function is what fits our IV curves for us, extracting the DC characteristics we talked about earlier, and detailed further in the DC characteristic section in the presentation. We should ask ourselves a few questions at this point:

1. Which region is superconducting, normal, or in transition on each graph?
2. Does the behavior of each curve match what we would expect in each region?
3. Do we have any strange points? (perhaps think about what would happen if we specified more superconduting points than there actually was)
4. What parameters do we have now? 

## Fit Rload and  Rn

In [6]:
IVobj.fit_rload_rn(lgcplot=True)

**fit_rload_rn** this function is self explanatory. 

## Fit dIdV (TES in transition)

THEORY OF SMALL SIGNAL RESPONSE HERE

In [7]:
IVobj.fit_tran_didv(lgcplot=True, lp_cutoff=20e3, zoomfactor=0.05)

**fit_tran_didv** this function fits the small signal response of our device, our dynamical characteristics. Here we also applied a low pass cut off filter just to clean up some of the fuzziness. 

1. Now what parameters do we have?

Theoretical we know how we should fit our small signal response parameters, but how is this done actually/analytically? What is the process behind the curtain of easy function calls which do heavy lifting? 

## Noise analysis

NOISE ANALYSIS THEORY HERE

In [10]:
IVobj.tc = 48.8e-3
IVobj.tbath = 4.5e-3
IVobj.tload = 65e-3
IVobj.Gta = 5 * IVobj.df[IVobj.didvinds].iloc[9]['ptes']/IVobj.tc

In [8]:
# noise modeling
IVobj.fit_normal_noise(fit_range = [1e2, 1e5], lgcplot=True, lgcsave=False)
IVobj.fit_sc_noise(fit_range = [1e2, 1e5], lgcplot=True, lgcsave=False)

# using  default collection efficiency = 1
IVobj.model_noise_simple(lgcplot=True, lgcsave=False, tau_collect=250e-6,collection_eff=1)

In [12]:
# noise errors
IVobj.estimate_noise_errors(tau_collect=0, collection_eff=1, nsamples=500)

## Find Optimum bias

In [9]:
IVobj.find_optimum_bias(lgcplot=True, ylims=[0, 0.4])

In [14]:
print("Bias (uA), R0 (mOhms), I0 (nA), PTES (fW)")

for index, row in IVobj.df[IVobj.didvinds].iloc[0:-1].iterrows():
    print(
        f"{row['qetbias'] * 1e6:.1f}     ",
        f"{row['r0'] * 1e3:.1f}       ",
        f"{row['i0'] * 1e9:.1f}    ",
        f"{row['ptes'] * 1e15:.1f}    ",
        f"{row['didvobj'].fitresult(2).get('params', {'tau1':0}).get('tau1')*1e6:.1f}",
        f"{row['didvobj'].fitresult(2).get('params', {'tau2':0}).get('tau2')*1e6:.1f}",
        f"{row['didvobj'].fitresult(3).get('params', {'tau1':0}).get('tau1')*1e6:.1f}",
        f"{row['didvobj'].fitresult(3).get('params', {'tau2':0}).get('tau2')*1e6:.1f}",
        f"{row['didvobj'].fitresult(3).get('params', {'tau3':0}).get('tau3')*1e6:.1f}",
        #f"{row['didvobj'].fitresult(3)['falltimes'][0] * 1e6:.1f}",
        #f"{row['didvobj'].fitresult(3)['falltimes'][1] * 1e6:.1f}",
        #f"{row['didvobj'].fitresult(3)['falltimes'][2] * 1e6:.1f}",
    )
    

Bias (uA), R0 (mOhms), I0 (nA), PTES (fW)
-80.0      326.6        -1192.1     464.1     0.0 0.0 0.0 0.0 0.0
-70.0      326.7        -1042.5     355.1     0.0 0.0 0.0 0.0 0.0
-60.0      326.9        -894.0     261.3     0.0 0.0 0.0 0.0 0.0
-50.0      327.0        -744.7     181.3     0.0 0.0 0.0 0.0 0.0
-40.0      326.3        -596.9     116.2     0.0 0.0 0.0 0.0 0.0
-35.0      323.7        -526.7     89.8     687.2 1.3 686.9 1.3 4.8
-30.0      309.7        -470.7     68.6     1575.6 0.5 546.9 0.7 1032.9
-28.0      295.1        -461.3     62.8     2808.0 0.5 466.1 0.8 499.3
-26.0      271.0        -464.6     58.5     -1732.1 0.8 -44720514461.8 1.1 731.4
-24.0      238.9        -483.7     55.9     -240.9 1.0 -301.2 1.5 389.7
-22.0      204.8        -515.2     54.4     -162.9 1.5 -197.2 2.0 292.7
-20.0      169.0        -561.7     53.3     -114.3 1.9 -136.6 2.5 186.7
-19.0      152.3        -589.6     52.9     -92.7 2.0 -106.4 2.8 167.7
-18.0      136.2        -621.2     52.6     -80.8 2.